<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Close-Prices-and-Volumes-consolidation-to-DataFrame" data-toc-modified-id="Close-Prices-and-Volumes-consolidation-to-DataFrame-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Close Prices and Volumes consolidation to DataFrame</a></span></li></ul></div>

**General Description**
- Opening, parsing and combining all loaded files with quote prices to single DataFrame have been done in this notebook
- Data loaded from csv-files from `data/quotes/..csv`
- Output DataFrame saved into `datasets/df_prices.csv`
- All operations of opening ~10k files and recording data to the DataFrame takes about 4 hours, that's why this part of the Project's code is put on separate notebook
- Commodities and Financial Statements Data will be used only on selected tickers (after clusterization) and not processed in this notebook
- Because this process can take up to 3 hours, it is recommended to not run this code, the DataFrame already prepared and loaded to GoogleDrive

In [1]:
# necessary modules
import numpy as np
import pandas as pd
import os
import json
import csv
import time
import warnings
from datetime import date as dt

warnings.filterwarnings('ignore')

### Close Prices and Volumes consolidation to DataFrame

In [2]:
# Merging all quote tables
# attention: this cell can takes up to 3 hours!

quotes_file_list = os.listdir('data/quotes')
ticker_list = [i[:-11] for i in quotes_file_list]
quotes_collection = dict.fromkeys(ticker_list)

for file, label in zip(quotes_file_list, ticker_list):
    path = f'data/quotes/{file}'
    with open(path, 'r') as f:
        df = pd.read_csv(f)
        df = df.drop(columns=['Unnamed: 0'], axis=1)
        quotes_collection[label] = df

progress_scanner_int = 1000
start_time = time.time()
internal_time = time.time()

for idx, key in enumerate(ticker_list): # будет заменено на dict values
    df = quotes_collection[key].copy()
    df_dates = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.set_index(df_dates, inplace=True)
    df_prices = pd.DataFrame(df['close'])
    df_prices = df_prices.rename(columns={'close': key})
    if idx==0:
        df_prices_raw = df_prices.copy()
    else:
        df_prices_raw = df_prices_raw.join(df_prices, on=None, how='outer', lsuffix='_')
        
    if idx%progress_scanner_int == 0:
        print(f'{idx} tables merged in {time.time()-internal_time} sec')
        print(50*'.')
        internal_time = time.time()

#print(f'tables for {len(df_volumes_raw)} companies loaded in {time.time()-start_time} sec')

with open('datasets/df_prices.csv', 'w', encoding='utf-8') as f:
    df_prices_raw.to_csv(f, encoding='utf-8', columns=df_prices_raw.columns)